# Credit Risk Resampling Techniques

In [358]:
import warnings
warnings.filterwarnings('ignore')

In [406]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# Read the CSV and Perform Basic Data Cleaning

In [360]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [361]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [362]:
df_encode = df.copy()

In [363]:
col_str = []
col_1 = []
for c in df_encode.columns:
    if not isinstance(df_encode[c][0], float):
        if len(df_encode[c].unique()) > 1:
            print(c)
            print(df_encode[c].unique())
            col_str.append(c)
        else:
            col_1.append(c)
            
col_str

home_ownership
['RENT' 'MORTGAGE' 'OWN' 'ANY']
verification_status
['Source Verified' 'Verified' 'Not Verified']
issue_d
['Mar-2019' 'Feb-2019' 'Jan-2019']
loan_status
['low_risk' 'high_risk']
initial_list_status
['w' 'f']
next_pymnt_d
['May-2019' 'Apr-2019']
application_type
['Individual' 'Joint App']


['home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'initial_list_status',
 'next_pymnt_d',
 'application_type']

In [364]:
df_encode.drop(col_1, axis = 1, inplace=True)

In [365]:
df_encode['verification_status'] = df_encode['verification_status'].apply(lambda x: 1 if x != 'Not Verified' else 0)
df_encode['initial_list_status'] = df_encode['initial_list_status'].apply(lambda x: 1 if x == 'w' else 0)
df_encode['application_type'] = df_encode['application_type'].apply(lambda x: 1 if x == 'Individual' else 0)
df_encode['loan_status'] = df_encode['loan_status'].apply(lambda x: 1 if x == 'high_risk' else 0)

In [366]:
df_encode = pd.get_dummies(df_encode, columns=["home_ownership", "issue_d", "next_pymnt_d"])
df_encode

,loan_amnt,int_rate,installment,annual_inc,verification_status,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
0,10500.0,0.1719,375.35,66000.0,1,0,27.24,0.0,0.0,8.0,...,61987.0,0,0,0,1,0,0,1,0,1
1,25000.0,0.2000,929.09,105000.0,1,0,20.23,0.0,0.0,17.0,...,49197.0,0,1,0,0,0,0,1,0,1
2,20000.0,0.2000,529.88,56000.0,1,0,24.26,0.0,0.0,8.0,...,43144.0,0,1,0,0,0,0,1,0,1
3,10000.0,0.1640,353.55,92000.0,1,0,31.44,0.0,1.0,10.0,...,76506.0,0,0,0,1,0,0,1,0,1
4,22000.0,0.1474,520.39,52000.0,0,0,18.76,0.0,1.0,14.0,...,20000.0,0,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,1,0,9.60,0.0,0.0,9.0,...,5425.0,0,0,0,1,0,1,0,0,1
68813,12000.0,0.2727,368.37,63000.0,0,0,29.07,0.0,0.0,8.0,...,62939.0,0,0,0,1,0,1,0,0,1
68814,5000.0,0.1992,185.62,52000.0,1,0,14.86,0.0,0.0,5.0,...,18492.0,0,1,0,0,0,1,0,0,1
68815,40000.0,0.0646,1225.24,520000.0,1,0,9.96,0.0,1.0,21.0,...,78634.0,0,1,0,0,0,1,0,0,1


In [367]:
for c in df_encode.columns:
    if isinstance(df_encode[c][0], float):
        print(c, df[c].max(), df[c].min())

loan_amnt 40000.0 1000.0
int_rate 0.3084 0.06
installment 1676.23 30.89
annual_inc 8797500.0 40.0
dti 999.0 0.0
delinq_2yrs 18.0 0.0
inq_last_6mths 5.0 0.0
open_acc 72.0 2.0
pub_rec 4.0 0.0
revol_bal 587191.0 0.0
total_acc 164.0 3.0
out_prncp 40000.0 0.0
out_prncp_inv 40000.0 0.0
total_pymnt 41418.9811651124 0.0
total_pymnt_inv 41418.98 0.0
total_rec_prncp 40000.0 0.0
total_rec_int 2735.4 0.0
total_rec_late_fee 75.24 0.0
recoveries 0.0 0.0
collection_recovery_fee 0.0 0.0
last_pymnt_amnt 40500.34 0.0
collections_12_mths_ex_med 4.0 0.0
policy_code 1.0 1.0
acc_now_delinq 0.0 0.0
tot_coll_amt 159199.0 0.0
tot_cur_bal 3140082.0 235.0
open_acc_6m 14.0 0.0
open_act_il 63.0 0.0
open_il_12m 6.0 0.0
open_il_24m 22.0 0.0
mths_since_rcnt_il 389.0 0.0
total_bal_il 1260281.0 1.0
il_util 313.0 0.0
open_rv_12m 18.0 0.0
open_rv_24m 44.0 0.0
max_bal_bc 207484.0 0.0
all_util 216.0 1.0
total_rev_hi_lim 759500.0 200.0
inq_fi 29.0 0.0
total_cu_tl 70.0 0.0
inq_last_12m 42.0 0.0
acc_open_past_24mths 45.0 0.0


# Split the Data into Training and Testing

In [368]:
# Create our Current
X = df_encode.drop(columns="loan_status")

# Create our target
y = df_encode[target]

In [369]:
# Check the balance of our target values
y['loan_status'].value_counts()

0    68470
1      347
Name: loan_status, dtype: int64

In [370]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y, train_size=0.80)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(55053, 88)
(13764, 88)
(55053, 1)
(13764, 1)


In [371]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [372]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE

In [373]:
Counter(y_train['loan_status'])

Counter({0: 54775, 1: 278})

In [374]:
Counter(y_test['loan_status'])

Counter({0: 13695, 1: 69})

In [375]:
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

In [376]:
Counter(y_resampled['loan_status'])

Counter({0: 54775, 1: 54775})

In [377]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [378]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [379]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [380]:
y_pred = model.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.8520778843359489

In [381]:
# Display the confusion matrix
# YOUR CODE HERE

In [382]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[11672  2023]
 [   13    56]]


In [383]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [384]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.85      0.92     13695
           1       0.03      0.81      0.05        69

    accuracy                           0.85     13764
   macro avg       0.51      0.83      0.49     13764
weighted avg       0.99      0.85      0.92     13764



### SMOTE Oversampling

In [385]:
# Resample the training data with SMOTE
# YOUR CODE HERE

In [389]:
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train_scaled, y_train)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [390]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [391]:
y_pred = model.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.8722755013077593

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [392]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[11954  1741]
 [   17    52]]


In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [393]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.87      0.93     13695
           1       0.03      0.75      0.06        69

    accuracy                           0.87     13764
   macro avg       0.51      0.81      0.49     13764
weighted avg       0.99      0.87      0.93     13764



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

In [395]:
cc = ClusterCentroids(random_state=1)
%timeit X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

In [396]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [397]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [398]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [399]:
y_pred = model.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.7585730892182505

In [400]:
# Display the confusion matrix
# YOUR CODE HERE

In [401]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[10388  3307]
 [   16    53]]


In [402]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [403]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.76      0.86     13695
           1       0.02      0.77      0.03        69

    accuracy                           0.76     13764
   macro avg       0.51      0.76      0.45     13764
weighted avg       0.99      0.76      0.86     13764



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [409]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

In [410]:
smote_enn = SMOTEENN(random_state=0)
%timeit X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)

4min 11s ± 8.25 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [411]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [412]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [413]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [414]:
y_pred = model.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.7585730892182505

In [415]:
# Display the confusion matrix
# YOUR CODE HERE

In [416]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[10388  3307]
 [   16    53]]


In [417]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [418]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.76      0.86     13695
           1       0.02      0.77      0.03        69

    accuracy                           0.76     13764
   macro avg       0.51      0.76      0.45     13764
weighted avg       0.99      0.76      0.86     13764

